In [23]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({'role':message["role"],'content':message["content"]})
    
    response = client.chat.completions.create(
        model = model_name,
        messages=input_messages,
        temperature= temperature, # Index of the response
        top_p=0.8, # Probability of the response
        max_tokens = 2000 # Maximum number of tokens (words) in the response
    ).choices[0].message.content
    
    return response
    

In [2]:
client = OpenAI(
    api_key = os.getenv('RUNPOD_TOKEN'),
    base_url= os.getenv('RUNPOD_CHATBOT_URL'),
    )
model_name = os.getenv("MODEL_NAME")

## GET LLM RESPONSE

# Promt Engineering

# Structed output

In [22]:
system_prompt = """
You are a helpful asisstant that answers questions about capital of countries.
Your output should be in a strctured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    "country": the country that you will get the capital of
    "capital": the capital of the country stated
}
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What's the capital of Germany?"})
response = get_chatbot_response(client,model_name,messages)

print(response)

[
{
    "country": "Germany",
    "capital": "Berlin"
}
]


In [24]:
type(response)

str

In [27]:
json_response = json.loads(response)
json_response

[{'country': 'Germany', 'capital': 'Berlin'}]

In [30]:
type(json_response[0]),json_response[0]["capital"]

(dict, 'Berlin')

# Input Structuring

In [33]:
user_input = """
Get me the capitals of the following countries:
```
1. Italy
2. Germany
3. France
```
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


In [34]:
json_response = json.loads(response)

# Give the model time to think(Chain of thought)

In [35]:
user_prompt = """
Calculate the result of this equation: 1+3

Your out put should be in a structered json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from the calculating the equation above
}
"""
messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [42]:
259/2*8654+91072*33-12971

4113098.0

In [43]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your out put should be in a structered json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from the calculating the equation above
}
"""
messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1344449
}


In [44]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your out put should be in a structered json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    step: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result. Feel free to write in free text.
    result: The final number resulted from the calculating the equation above
}
"""
messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "step": "First, calculate the division: 259/2 = 129.5. Then, multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, multiply 91072 by 33: 91072 * 33 = 3,003,336. Now, add 112,141.5 and 3,003,336: 112,141.5 + 3,003,336 = 3,115,477.5. Finally, subtract 12971 from 3,115,477.5: 3,115,477.5 - 12971 = 3,105,706.5.",
    "result": 3105706.5
}


# RAG

In [ ]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

There is no such thing as an iPhone 16. Apple has not released an iPhone with that model number. The latest iPhone models available in the market are iPhone 13 series, which includes iPhone 13, iPhone 13 Mini, iPhone 13 Pro, and iPhone 13 Pro Max.

However, rumors and leaks about the upcoming iPhone 14 series have been circulating online. According to reports, the iPhone 14 series is expected to feature several upgrades and improvements, including:

1. New design: The iPhone 14 series is expected to have a new design with a smaller notch, thinner bezels, and a more seamless glass front.
2. Improved cameras: The iPhone 14 Pro models are expected to feature a new triple-camera setup with a wider-angle lens, a telephoto lens, and a ultra-wide-angle lens.
3. Faster processors: The iPhone 14 series is expected to be powered by Apple's A15 Bionic chip, which will provide faster performance and improved power efficiency.
4. Longer battery life: The iPhone 14 series is expected to have longer 

In [49]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [52]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger displays: 6.1-inch for the base model, 6.7-inch for the iPhone 16 Plus, 6.3 inches for the iPhone 16 Pro, and 6.9 inches for the iPhone 16 Pro Max.
2. Thinner bezels and a more durable Ceramic Shield.
3. New A18 chip (A18 Pro for the Pro models) with improved performance, neural engine capabilities, and faster GPU for gaming and machine learning tasks.
4. Upgraded camera systems:
	* Base iPhone 16: dual-camera setup with a 48MP main sensor.
	* Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button for more flexible photography options.
5. Advanced audio features, including "Audio Mix" for refined audio capture during video recording.
6. Extended battery life, especially in the iPhone 16 Pro Max, which has the longest-lasting battery of any iPhone.
7. Switch to USB-C for faster charging and data transfer.
8. Pro models support up to 2x faster video encoding.
9. Sta